<a href="https://colab.research.google.com/github/rajagopalmotivate/AIforEmpoweringPersonswithDisability/blob/main/BKB_ImageBind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Clone ImageBind from GitHub, download test files, and install dependencies



In [1]:
!git clone https://github.com/facebookresearch/ImageBind.git
!ln -s ./ImageBind/bpe/ .

Cloning into 'ImageBind'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 146 (delta 62), reused 50 (delta 40), pack-reused 47 (from 1)
Receiving objects: 100% (146/146), 2.64 MiB | 3.99 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
!mkdir -p test_data
!wget -P test_data --quiet https://github.com/BotsKnowBest/ImageBind/raw/main/test_data/lotr_boro.wav
!wget -P test_data --quiet https://github.com/BotsKnowBest/ImageBind/raw/main/test_data/startup.wav

In [3]:
!cd ImageBind && pip install -r requirements.txt


  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-xj19jp14/pytorchvideo_57354baea51645119af7b7b21c44a934
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-xj19jp14/pytorchvideo_57354baea51645119af7b7b21c44a934
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 

###Import packages

In [4]:
%matplotlib inline

import numpy as np
import torch
import torch.nn.functional as F
import sys
import warnings
from pathlib import Path

import ipywidgets as widgets
import IPython
from IPython.display import Image
from tqdm.notebook import tqdm

In [7]:
from imagebind import data


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [8]:

import imagebind.data as ib_data

In [12]:
from imagebind.models import imagebind_model


In [13]:
from imagebind.models.imagebind_model import ModalityType

In [10]:
sys.path.insert(0,'./ImageBind/')


In [11]:
sys.path

['./ImageBind/',
 './ImageBind/',
 './ImageBind/',
 './ImageBind/',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/root/.ipython']

In [9]:
sys.path.insert(0,'./ImageBind/')

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)

#import data as ib_data

#from models import imagebind_model
#from models.imagebind_model import ModalityType

ModuleNotFoundError: No module named 'models'

### Instantiate ImageBind model

In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

ib_model = imagebind_model.imagebind_huge(pretrained=True)
ib_model.eval()
ib_model.to(device)

print(f"ImageBind model loaded to {device}.")

  0%|          | 0.00/4.47G [00:00<?, ?B/s]

ImageBind model loaded to cuda:0.


In [17]:
test_text_list = ["A dog.", "A car", "A bird"]
test_image_paths = ["./ImageBind/.assets/dog_image.jpg", "./ImageBind/.assets/car_image.jpg", "./ImageBind/.assets/bird_image.jpg"]
test_audio_paths = ["./ImageBind/.assets/dog_audio.wav", "./ImageBind/.assets/car_audio.wav", "./ImageBind/.assets/bird_audio.wav"]

test_inputs = {
    ModalityType.TEXT: ib_data.load_and_transform_text(test_text_list, device),
    ModalityType.VISION: ib_data.load_and_transform_vision_data(test_image_paths, device),
    ModalityType.AUDIO: ib_data.load_and_transform_audio_data(test_audio_paths, device),
}

In [18]:
with torch.no_grad():
    test_embeddings = ib_model(test_inputs)

In [20]:

test_vt = torch.softmax(test_embeddings[ModalityType.VISION] @ test_embeddings[ModalityType.TEXT].T, dim=-1)
test_at = torch.softmax(test_embeddings[ModalityType.AUDIO] @ test_embeddings[ModalityType.TEXT].T, dim=-1)
test_va = torch.softmax(test_embeddings[ModalityType.VISION] @ test_embeddings[ModalityType.AUDIO].T, dim=-1)

In [21]:
print(f"Vision x Text: {test_vt}")
print(f"Audio x Text: {test_at}")
print(f"Vision x Audio: {test_va}")

Vision x Text: tensor([[9.9761e-01, 2.3694e-03, 1.8613e-05],
        [3.3837e-05, 9.9994e-01, 2.4118e-05],
        [4.7997e-05, 1.3496e-02, 9.8646e-01]], device='cuda:0')
Audio x Text: tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], device='cuda:0')
Vision x Audio: tensor([[0.8070, 0.1088, 0.0842],
        [0.1036, 0.7884, 0.1079],
        [0.0018, 0.0022, 0.9960]], device='cuda:0')


### Check that ImageBind inference works

In [22]:
test_text_list = ["A dog.", "A car", "A bird"]
test_image_paths = ["./ImageBind/.assets/dog_image.jpg", "./ImageBind/.assets/car_image.jpg", "./ImageBind/.assets/bird_image.jpg"]
test_audio_paths = ["./ImageBind/.assets/dog_audio.wav", "./ImageBind/.assets/car_audio.wav", "./ImageBind/.assets/bird_audio.wav"]

test_inputs = {
    ModalityType.TEXT: ib_data.load_and_transform_text(test_text_list, device),
    ModalityType.VISION: ib_data.load_and_transform_vision_data(test_image_paths, device),
    ModalityType.AUDIO: ib_data.load_and_transform_audio_data(test_audio_paths, device),
}

with torch.no_grad():
    test_embeddings = ib_model(test_inputs)

test_vt = torch.softmax(test_embeddings[ModalityType.VISION] @ test_embeddings[ModalityType.TEXT].T, dim=-1)
test_at = torch.softmax(test_embeddings[ModalityType.AUDIO] @ test_embeddings[ModalityType.TEXT].T, dim=-1)
test_va = torch.softmax(test_embeddings[ModalityType.VISION] @ test_embeddings[ModalityType.AUDIO].T, dim=-1)

print(f"Vision x Text: {test_vt}")
print(f"Audio x Text: {test_at}")
print(f"Vision x Audio: {test_va}")


Vision x Text: tensor([[9.9761e-01, 2.3694e-03, 1.8613e-05],
        [3.3837e-05, 9.9994e-01, 2.4118e-05],
        [4.7997e-05, 1.3496e-02, 9.8646e-01]], device='cuda:0')
Audio x Text: tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], device='cuda:0')
Vision x Audio: tensor([[0.8070, 0.1088, 0.0842],
        [0.1036, 0.7884, 0.1079],
        [0.0018, 0.0022, 0.9960]], device='cuda:0')


### Download ImageNetV2Dataset

In [23]:
!pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

  Cloning https://github.com/modestyachts/ImageNetV2_pytorch to /tmp/pip-req-build-p61g8hor
  Running command git clone --filter=blob:none --quiet https://github.com/modestyachts/ImageNetV2_pytorch /tmp/pip-req-build-p61g8hor
  Resolved https://github.com/modestyachts/ImageNetV2_pytorch to commit 14d4456c39fe7f02a665544dd9fc37c1a5f8b635
  Preparing metadata (setup.py) ... done
  Created wheel for imagenetv2_pytorch: filename=imagenetv2_pytorch-0.1-py3-none-any.whl size=2657 sha256=c2d43432ca206d609c24267d0c707c6502ace41e9127af733865116ac53aadfe
  Stored in directory: /tmp/pip-ephem-wheel-cache-_wke5w5b/wheels/ea/e3/2d/38c8d17086a0ea5890dc0d4796db505e41323d2d9800b56fa7
Successfully built imagenetv2_pytorch


In [24]:
from imagenetv2_pytorch import ImageNetV2Dataset

images = ImageNetV2Dataset()
print(f"{len(ImageNetV2Dataset().fnames)} images found.")

Dataset matched-frequency not found on disk, downloading....


100%|██████████| 1.26G/1.26G [00:51<00:00, 24.5MiB/s]


Extracting....
10000 images found.


### Compute ImageBind embeddings for ImageNetV2Dataset

In [25]:
def chunker(seq, size):
    return (seq[pos:pos+size] for pos in range(0, len(seq), size))

def get_load_and_transform_fn(modality):
    if modality in [ModalityType.VISION, ModalityType.THERMAL, ModalityType.DEPTH, ModalityType.IMU]:
        return ib_data.load_and_transform_vision_data
    elif modality == ModalityType.TEXT:
        return ib_data.load_and_transform_text
    elif modality == ModalityType.AUDIO:
        return ib_data.load_and_transform_audio_data
    elif modality == 'video':
        return ib_data.load_and_transform_video_data

def get_dataset_imagebin_embeddings(paths_batches, modality, batch_size=32):
    assert modality in vars(ModalityType).values()

    ib_embeds_fnames = []
    ib_embeds = []
    with torch.no_grad():
        fnames_batch = list(chunker(paths_batches, batch_size))
        for i, fnames_batch in enumerate(tqdm(fnames_batch)):
            inputs = {
                modality: get_load_and_transform_fn(modality)(fnames_batch, device)
            }
            ib_embed = ib_model(inputs)[modality]

            ib_embeds_fnames.extend(fnames_batch)
            ib_embeds.append(ib_embed)

    ib_embeds = torch.vstack(ib_embeds)
    return {
        'embeds': ib_embeds,
        'fnames': ib_embeds_fnames
    }


In [40]:
imagenetv2_images_paths = [str(n) for n in ImageNetV2Dataset().fnames]


In [41]:
imagenetv2_images_paths2 = imagenetv2_images_paths[:40]

In [42]:
imagenetv2_images_paths = imagenetv2_images_paths2

In [43]:
#imagenetv2_images_paths = [str(n) for n in ImageNetV2Dataset().fnames]
ibembeds_dataset_vision = get_dataset_imagebin_embeddings(imagenetv2_images_paths, ModalityType.VISION)

print(ibembeds_dataset_vision['embeds'].shape)

  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([40, 1024])


### Find 5 closest images

In [27]:
def get_closest_embeds(input_data, dataset_embeds):
    # Normalize dataset embeddings
    dataset_embeds_norm = dataset_embeds / dataset_embeds.norm(dim=-1, keepdim=True)
    dataset_embeds_norm = dataset_embeds_norm.cpu().numpy()

    # Pre-process inputs
    inputs = {}
    for modality in input_data:
        assert modality in vars(ModalityType).values()
        inputs[modality] = get_load_and_transform_fn(modality)([input_data[modality]], device)

    # Compute embeds
    with torch.no_grad():
        embeddings = ib_model(inputs)

    # Find closest embeddings
    closest_embeds = {}
    for modality in vars(ModalityType).values():
        if modality in input_data:
            vec_norm = embeddings[modality] / embeddings[modality].norm(dim=-1, keepdim=True)
            vec_norm = vec_norm.cpu().numpy()

            similarities = vec_norm @ dataset_embeds_norm.T
            closest_embeds[modality] = list(reversed(np.argsort(similarities, axis=1).tolist()[0]))

    return closest_embeds

In [28]:
def display_closest_images(input_data, dataset_vision, top_k=1):
    if dataset_vision:
        closest_embeds_vision = get_closest_embeds(input_data, dataset_vision['embeds'])

    assert top_k >= 1

    for modality in input_data:
        assert modality in vars(ModalityType).values()

        # Display image
        if dataset_vision:
            images_to_display = []
            for n in range(top_k):
                closest_image_idx = closest_embeds_vision[modality][n]
                closest_image_fname = dataset_vision['fnames'][closest_image_idx]


                img = open(closest_image_fname,'rb').read()
                images_to_display.append(widgets.Image(value=img, width=190))

            wid=widgets.HBox(images_to_display)
            display(wid)


In [44]:
input_data = {
    ModalityType.AUDIO: "test_data/startup.wav",
}

res = display_closest_images(
    input_data,
    dataset_vision=ibembeds_dataset_vision,
    top_k=5
)

In [45]:
input_data = {
    ModalityType.AUDIO: "test_data/lotr_boro.wav",
}

res = display_closest_images(
    input_data,
    dataset_vision=ibembeds_dataset_vision,
    top_k=5
)